## FRI Ограничения

Алгоритм FRI (Fast Reed-Solomon Interactive Oracle Proofs of Proximity) является ключевым компонентом системы доказательства нулевого разглашения (Zero-Knowledge) в контексте STARKs (Scalable Transparent Arguments of Knowledge). Он используется для проверки того, что функция является многочленом низкой степени (или близка к нему) с использованием минимального количества вычислений и взаимодействий.

FRI играет важную роль в сокращении размера доказательств и повышения их эффективности, что особенно важно для блокчейн-приложений.

Подгружаем переменные для второй части:

In [2]:
from channels.channel import Channel
from fields.field import FieldElement
from merkles.merkle import MerkleTree
from polynomials.polynomial import Polynomial
from sessions.sessions import part2

cp, cp_eval, cp_merkle, channel, eval_domain = part2()
print("Успешно!")

100%|██████████| 1023/1023 [00:04<00:00, 205.93it/s]


Успешно!


#### FRI упаковка

Наша цель в этой части – построить слои FRI и выполнить на них фиксацию.
Для получения каждого слоя нам необходимо:

1) Сгенерировать домен для слоя (из домена предыдущего слоя).
2) Сгенерировать полином для слоя (из полинома и домена предыдущего слоя).
3) Оценить указанный полином на указанном домене - это следующий слой FRI.

#### Domain Generation

Первый домен FRI - это просто eval_domain, который вы уже сгенерировали в первой части, а именно косет группы порядка 8192. Каждый последующий домен FRI получается путем взятия первой половины предыдущего домена FRI (отбрасывая вторую половину) и возведения в квадрат каждого из его элементов.

Формально - мы получили eval_domain, взяв:
$$w,w * h, w * h^2, ... , w * h^{8191}$$

Следующий слой будет выглядить примерно вот так:
$$w^2,(w*h)^2,(w*h^2)^2, ..., (w * h^{4095})^2$$

Обратите внимание, что взятие квадратов второй половины каждого элемента в eval_domain дает точно такой же результат, как и взятие квадратов первой половины. Это справедливо и для следующих слоев. Например:

In [3]:
print(eval_domain[100] ** 2)
half_domain_size = len(eval_domain) // 2
print(eval_domain[half_domain_size + 100] ** 2)

-373161870
-373161870


Аналогично, домен третьего слоя будет:
$$w^2,(w*h)^4,(w*h^2)^4, ..., (w * h^{2047})^4$$

Напишем функцию next_fri_domain, которая принимает в качестве аргумента предыдущий домен и выводит следующий.

In [4]:
def next_fri_domain(fri_domain):
    return [x ** 2 for x in fri_domain[:len(fri_domain) // 2]]

In [5]:
# Проверка на предварительно вычисленный хэш.
from hashlib import sha256
next_domain = next_fri_domain(eval_domain)
assert '5446c90d6ed23ea961513d4ae38fc6585f6614a3d392cb087e837754bfd32797' == sha256(','.join([str(i) for i in next_domain]).encode()).hexdigest()
print('Успешно!')

Успешно!


#### FRI упаковочный оператор

Первый полином FRI - это просто полином композиции, т.е. $cp$.
Каждый последующий полином FRI получается путем:

1) Получения элемента случайного поля $\beta$ (вызовом Channel.receive_random_field_element).
2) Умножения нечетных коэффициентов предыдущего полинома на $\beta$.
3) Суммирования последовательных пар (четных-нечетных) коэффициентов.


Формально, пусть $k$-й полином имеет степень $< m$ (где $m$ — это некоторая степень двойки):

$$
p_k(x) := \sum_{i=0}^{m-1} c_i x^i
$$

Тогда $(k+1)$-й полином, степень которого $< \frac{m}{2}$, определяется как:

$$
p_{k+1}(x) := \sum_{i=0}^{m/2 - 1} \left( c_{2i} + \beta \cdot c_{2i+1} \right) x^i
$$

Где $\beta$ — это определенный коэффициент, зависящий от рандомизации процесса.

Напишим функцию next_fri_polynomial, которая принимает в качестве аргументов полином и элемент поля (тот, который мы назвали $\beta$), и возвращает «сложенный» следующий полином.

Polynomial.poly содержит список коэффициентов полинома, в котором свободный член стоит первым, а высшая степень - последней, поэтому p.poly[i] == u, если коэффициент $x^i$ это $u^*$

In [6]:
def next_fri_polynomial(poly,  beta):
    odd_coefficients = poly.poly[1::2]
    even_coefficients = poly.poly[::2]
    odd = beta * Polynomial(odd_coefficients)
    even = Polynomial(even_coefficients)
    return odd + even

In [7]:
next_p = next_fri_polynomial(cp, FieldElement(987654321))
assert '6bff4c35e1aa9693f9ceb1599b6a484d7636612be65990e726e52a32452c2154' == sha256(','.join([str(i) for i in next_p.poly]).encode()).hexdigest()
print('Успешно!')

Успешно!


#### Соберем все вместе, чтобы получить следующий слой FRI

Напишем функцию next_fri_layer, которая принимает многочлен, домен и элемент поля (снова - $\beta$) и возвращает следующий многочлен, следующий домен и оценку этого следующего многочлена на этом следующем домене.


In [8]:
def next_fri_layer(poly, domain, beta):
    next_poly = next_fri_polynomial(poly, beta)
    next_domain = next_fri_domain(domain)
    next_layer = [next_poly(x) for x in next_domain]
    return next_poly, next_domain, next_layer

In [9]:
test_poly = Polynomial([FieldElement(2), FieldElement(3), FieldElement(0), FieldElement(1)])
test_domain = [FieldElement(3), FieldElement(5)]
beta = FieldElement(7)
next_p, next_d, next_l = next_fri_layer(test_poly, test_domain, beta)
assert next_p.poly == [FieldElement(23), FieldElement(7)]
assert next_d == [FieldElement(9)]
assert next_l == [FieldElement(86)]
print('Успешно!')

Успешно!


#### Генерация обязательств FRI

Теперь мы разработали инструменты для написания метода FriCommit, который содержит основной цикл создания обязательств FRI.

Он принимает следующие 5 аргументов:

1) Полином композиции, который также является первым полиномом FRI, то есть - cp.
2) Coset порядка 8192, который также является первым доменом FRI, то есть - eval_domain.
3) Оценка первого над вторым, которая также является первым слоем FRI, то есть - cp_eval.
4) Первое дерево Меркла (у нас будет по одному для каждого слоя FRI), построенное из этих оценок, то есть - cp_merkle.
5) Объект канала, то есть channel.

Соответственно, метод возвращает 4 списка:

1) Полиномы FRI.
2) Домены FRI.
3) Слои FRI.
4) Деревья Меркла FRI.

Метод содержит цикл, в каждой итерации которого мы расширяем эти четыре списка, используя последний элемент в каждом. Итерация должна остановиться, когда последний полином FRI будет иметь степень 0, то есть когда последний полином FRI будет просто константой. Затем следует отправить по каналу эту константу (т.е. свободный член многочлена). Класс Channel поддерживает только отправку строк, поэтому перед отправкой обязательно преобразуйте все, что вы хотите отправить по каналу, в строку.

In [10]:
def FriCommit(cp, domain, cp_eval, cp_merkle, channel):    
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    while fri_polys[-1].degree() > 0:
        beta = channel.receive_random_field_element()
        next_poly, next_domain, next_layer = next_fri_layer(fri_polys[-1], fri_domains[-1], beta)
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        fri_merkles.append(MerkleTree(next_layer))
        channel.send(fri_merkles[-1].root)   
    channel.send(str(fri_polys[-1].poly[0]))
    return fri_polys, fri_domains, fri_layers, fri_merkles

In [11]:
test_channel = Channel()
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, test_channel)
assert len(fri_layers) == 11, f'Ожидаемое количество слоев FRI равно 11, тогда как на самом деле оно равно {len(fri_layers)}.'
assert len(fri_layers[-1]) == 8, f'Ожидается, что последний слой содержит ровно 8 элементов, он содержит {len(fri_layers[-1])}.'
assert all([x == FieldElement(-1138734538) for x in fri_layers[-1]]), f'Ожидается, что последний слой будет постоянным.'
assert fri_polys[-1].degree() == 0, 'Расшифровка последнего многочлена как постоянного (степень 0).'
assert fri_merkles[-1].root == '1c033312a4df82248bda518b319479c22ea87bd6e15a150db400eeff653ee2ee', 'Корень Merkle последнего слоя неверен.'
assert test_channel.state == '61452c72d8f4279b86fa49e9fb0fdef0246b396a4230a2bfb24e2d5d6bf79c2e', 'Состояние канала не соответствует ожиданиям.'
print('Успешно!')

Успешно!


Вывод доказательства:

In [12]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)
print(channel.proof) 

['send:6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04', 'receive_random_field_element:2948900820', 'receive_random_field_element:1859037345', 'receive_random_field_element:2654806830', 'send:61f7d8283e244d391a483c420776e351fcfdbce525a698461a8307a1345b5652', 'receive_random_field_element:394024765', 'send:9431516ee735a498c4aec3da30112e417b03e55e5be939ff44ca8a0a62475b15', 'receive_random_field_element:1705983878', 'send:584b4b88a7f296efa0309d8e6faef13573b1ee5dfcb02ed8be5d853172f3fc69', 'receive_random_field_element:665918954', 'send:2debb983bb6473a5d4e9046944fb7ef66ef814c64f58ca5d8ebc2a15ed61ca4a', 'receive_random_field_element:3182659911', 'send:5da75aa9d9a9a564d7f19e431cbbb91eff030c353f3825dc5352674d1b7813f9', 'receive_random_field_element:2692084106', 'send:8ca6b618f3d758e7a99c4988e3a30e5c443f6f4ed79c64b698b031cca67ee4c2', 'receive_random_field_element:2453626065', 'send:db00ee380f0b1a9c2aa37efe2eabca468a83370046cf824eea9409e536256996', 'receive_random_field_element: